In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, KBinsDiscretizer
from deep_logic.models.brl import XBRLClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader
import time
from sklearn.metrics import accuracy_score
from sympy import simplify_logic
from sklearn.model_selection import train_test_split

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relu_nn import get_reduced_model, prune_features
from deep_logic.utils.psi_nn import prune_equal_fanin
from deep_logic import logic
import deep_logic as dl

set_seed(0)

#%%

data = pd.read_csv('data/mimic-ii/full_cohort_data.csv')
# data.drop('hgb_first')
fs = [
    'aline_flg',
    'gender_num',
    # 'hosp_exp_flg',
    # 'icu_exp_flg',
    # 'day_28_flg',
    # 'censor_flg',
    'sepsis_flg', 'chf_flg', 'afib_flg',
    'renal_flg', 'liver_flg', 'copd_flg', 'cad_flg', 'stroke_flg',
    'mal_flg', 'resp_flg',
]
features = fs
data1 = data[fs].values
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
data1 = imp_mean.fit_transform(data1)

f2 = fs.copy()
f2.append('day_icu_intime')
f2.append('service_unit')
f2.append('day_28_flg')
f2.append('hospital_los_day')
f2.append('icu_exp_flg')
f2.append('hosp_exp_flg')
f2.append('censor_flg')
f2.append('mort_day_censored')
f2 = data.columns.difference(f2)
data2 = data[f2].values
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
data2 = imp_mean.fit_transform(data2)
scaler = MinMaxScaler((0, 1))
data2 = scaler.fit_transform(data2)
features = features + list(f2)
est = KBinsDiscretizer(n_bins=3, encode='onehot-dense', strategy='uniform')
data2d = est.fit_transform(data2)
f2d = []
for feature in f2:
    #f2d.append(feature + '_VLOW')
    f2d.append(feature + '_LOW')
    f2d.append(feature + '_NORMAL')
    f2d.append(feature + '_HIGH')
    #f2d.append(feature + '_VHIGH')
features = fs + f2d

datax = np.hstack((data1, data2d))
datay = data['day_28_flg'].values

x = torch.FloatTensor(datax)
y = torch.LongTensor(datay)
print(x.shape)
print(y.shape)


torch.Size([1776, 90])
torch.Size([1776])


In [2]:
results_dir = 'results/mimic2'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# concepts = [f'c{i:03}' for i in range(x.shape[1])]
concepts =  features
n_rep = 10
tot_epochs = 5001
prune_epochs = 2001
seed = 42

n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [3]:
def train_nn(x_train, y_train, need_pruning, seed, device, l1=0.001, lr=0.001, relu=False, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 100),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 50),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 30),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(30, 10),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)
        
        if need_pruning or relu:
            for module in model.children():
                if isinstance(module, torch.nn.Linear):
                    loss += l1 * torch.norm(module.weight, 1)
                    loss += l1 * torch.norm(module.bias, 1)
                    break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch == prune_epochs and need_pruning and not relu and epoch % 1000 == 1:
            print('pruning')
            prune_features(model, n_classes=1, device=device)
            need_pruning = True
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [4]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [5]:
def c_to_y(method, need_pruning, l1=0.0001, lr=0.0001, relu=False, verbose=False):
    methods = []
    splits = []
    explanations = []
    explanations_inv = []
    model_accuracies = []
    explanation_accuracies = []
    explanation_accuracies_inv = []
    elapsed_times = []
    elapsed_times_inv = []
    explanation_fidelities = []
    explanation_complexities = []
    for split, (trainval_index, test_index) in enumerate(skf.split(x.cpu().detach().numpy(), y.cpu().detach().numpy())):
        print(f'Split [{split+1}/{n_splits}]')
        x_trainval, x_test = torch.FloatTensor(x[trainval_index]), torch.FloatTensor(x[test_index])
        y_trainval, y_test = torch.LongTensor(y[trainval_index]), torch.LongTensor(y[test_index])
        x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.3, random_state=42)
    
        explanation, explanation_inv = '', ''
        explanation_accuracy, explanation_accuracy_inv = 0, 0

        target_class = 1
        start = time.time()
            
        if method == 'tree':
            classifier = DecisionTreeClassifier(random_state=seed)
            classifier.fit(x_trainval.cpu().detach().numpy(), y_trainval.cpu().detach().numpy())
            y_preds = classifier.predict(x_test.cpu().detach().numpy())
            
            # extract formula
            explanation = tree_to_formula(classifier, concepts, target_class)
            
            # compute metrics
            model_accuracy = accuracy_score(y_test, y_preds)
            explanation_accuracy = model_accuracy
            explanation_fidelity = accuracy_score(y_preds, y_preds)
            explanation_complexity = dl.logic.complexity(explanation)
        
        elif method == 'brl':
            train_dataset = TensorDataset(x_trainval, y_trainval)
            val_dataset = TensorDataset(x_val, y_val)
            test_dataset = TensorDataset(x_test, y_test)
            dummy_concepts = [f'feature{i:010}' for i in range(len(concepts))]
            model = XBRLClassifier(name=os.path.join(results_dir, f'{method}_{split}'), 
                                   n_classes=1,#len(y_train.unique()), 
                                   n_features=x_train.shape[1], feature_names=dummy_concepts, 
                                   class_names=['~day_28_flg', 'day_28_flg'], discretize=True)
            results = model.fit(train_dataset, metric=accuracy_score, save=True)
            global_explanation, elapsed_time = model.get_global_explanation(0, concepts=dummy_concepts, return_time=True)
            explanation_fidelity = 0
            explanation_complexity = 0
            explanation = logic.base.replace_names(global_explanation, concepts)
            
            # compute metrics
            y_preds = model.forward(x_test)
            model_accuracy = accuracy_score(y_test, y_preds)
            target_class = 1
            explanation_accuracy, y_formula = logic.base.test_explanation(global_explanation, 
                                                                          target_class, 
                                                                          x=x_test, y=y_test,
                                                                          metric=accuracy_score)
            explanation_fidelity = accuracy_score(y_formula, y_formula)
            explanation_complexity = dl.logic.complexity(global_explanation)
                
        else:
            if method == 'psi':
                # positive class
                target_class = 1
                model = train_psi_nn(x_trainval, y_trainval.eq(target_class), need_pruning, seed, device, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()>0.5
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds)
                
            else:
                model = train_nn(x_trainval, y_trainval, need_pruning, seed, device, l1, lr, relu, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy().argmax(axis=1)
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds)
            
            print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
            
            # positive class
            target_class = 1
            if method == 'psi':
                global_explanation = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation, _, _ = logic.relu_nn.combine_local_explanations(model, 
                                                                                   x_val.to(device), 
                                                                                   y_val.to(device), 
                                                                                   topk_explanations=1,
                                                                                   target_class=target_class,
                                                                                   method=method, device=device)
            
            explanation_fidelity = 0
            explanation_complexity = 0
            if global_explanation:
                explanation = logic.base.replace_names(global_explanation, concepts)
                explanation_accuracy, y_formula = logic.base.test_explanation(global_explanation, 
                                                                              target_class, 
                                                                              x=x_test, y=y_test,
                                                                              metric=accuracy_score)
                explanation_fidelity = accuracy_score(y_preds, y_formula)
                explanation_complexity = dl.logic.complexity(global_explanation)

        elapsed_time = time.time() - start
        
        if verbose:
            print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
            print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
            print(f'\t Elapsed time {elapsed_time}')
            print(f'\t Fidelity: "{explanation_fidelity:.4f}" - Complexity: "{explanation_complexity}"')

        methods.append(method)
        splits.append(split)
        explanations.append(explanation)
        model_accuracies.append(model_accuracy)
        explanation_accuracies.append(explanation_accuracy)
        elapsed_times.append(elapsed_time)
        explanation_fidelities.append(explanation_fidelity)
        explanation_complexities.append(explanation_complexity)
    
    explanation_consistency = dl.logic.formula_consistency(explanations)
    print(f'Consistency of explanations: {explanation_consistency:.4f}')
    
    results = pd.DataFrame({
        'method': methods,
        'split': splits,
        'explanation': explanations,
        'model_accuracy': model_accuracies,
        'explanation_accuracy': explanation_accuracies,
        'explanation_fidelity': explanation_fidelities,
        'explanation_complexity': explanation_complexities,
        'explanation_consistency': explanation_consistency,
        'elapsed_time': elapsed_times,
    })
    results.to_csv(os.path.join(results_dir, f'results_{method}.csv'))
    
    return results

# General pruning

In [6]:
method = 'pruning'
need_pruning = True
relu = False
results_pruning = c_to_y(method, need_pruning, relu=relu, l1=0.000001, lr=0.0005, verbose=True)
results_pruning

Split [1/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.9099
	 Epoch 1500: train accuracy: 0.9124
	 Epoch 2000: train accuracy: 0.9124
pruning
Pruned 86/90 features
	 Epoch 2500: train accuracy: 0.8692
	 Epoch 3000: train accuracy: 0.8692
	 Epoch 3500: train accuracy: 0.8692
	 Epoch 4000: train accuracy: 0.8692
	 Epoch 4500: train accuracy: 0.8692
	 Epoch 5000: train accuracy: 0.8692
	 Model's accuracy: 0.8539
	 Model's accuracy: 0.8539
	 Class 1 - Global explanation: "stroke_flg & age_HIGH & ~afib_flg & ~sapsi_first_HIGH" - Accuracy: 0.8483
	 Elapsed time 15.94326114654541
	 Fidelity: "0.9719" - Complexity: "4"
Split [2/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.9055
	 Epoch 1500: train accuracy: 0.9093
	 Epoch 2000: train accuracy: 0.9099
pruning
Pruned 85/90 features
	 Epoch 2500: train accuracy: 0.8686
	 Epoch 3000: train accuracy: 0.8686
	 Epoch 3500: train acc

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,pruning,0,stroke_flg & age_HIGH & ~afib_flg & ~sapsi_fir...,0.853933,0.848315,0.971910,4,0.51,15.943261
1,pruning,1,stroke_flg & age_HIGH & ~afib_flg & ~hgb_first...,0.876404,0.859551,0.983146,5,0.51,12.870964
2,pruning,2,stroke_flg & age_HIGH & ~afib_flg & ~hgb_first...,0.876404,0.865169,0.966292,6,0.51,13.037436
3,pruning,3,stroke_flg & age_HIGH & ~afib_flg,0.865169,0.842697,0.943820,3,0.51,12.987042
4,pruning,4,stroke_flg & age_HIGH & ~afib_flg & ~hgb_first...,0.859551,0.853933,0.994382,5,0.51,12.822386
5,pruning,5,afib_flg & stroke_flg & age_HIGH & ~bun_first_...,0.870787,0.820225,0.904494,7,0.51,12.526067
6,pruning,6,stroke_flg & age_HIGH & ~afib_flg & ~hgb_first...,0.847458,0.836158,0.954802,5,0.51,13.439902
7,pruning,7,stroke_flg & age_HIGH & ~afib_flg & ~sapsi_fir...,0.870056,0.847458,0.977401,6,0.51,13.021081
8,pruning,8,stroke_flg & age_HIGH & ~afib_flg & ~bun_first...,0.870056,0.864407,0.949153,5,0.51,13.203390
9,pruning,9,stroke_flg & age_HIGH & ~afib_flg & ~map_1st_H...,0.870056,0.870056,0.977401,5,0.51,13.347927


# ReLUs

In [7]:
method = 'weights'
need_pruning = False
relu = True
results_weights = c_to_y(method='weights', need_pruning=False, l1=0.000001, lr=0.0005, relu=True, verbose=True)
results_weights

Split [1/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.9212
	 Epoch 1000: train accuracy: 0.9249
	 Epoch 1500: train accuracy: 0.9249
	 Epoch 2000: train accuracy: 0.9268
	 Epoch 2500: train accuracy: 0.9280
	 Epoch 3000: train accuracy: 0.9280
	 Epoch 3500: train accuracy: 0.9280
	 Epoch 4000: train accuracy: 0.9280
	 Epoch 4500: train accuracy: 0.9280
	 Epoch 5000: train accuracy: 0.9280
	 Model's accuracy: 0.8483
	 Model's accuracy: 0.8483
	 Class 1 - Global explanation: "afib_flg & resp_flg & ~stroke_flg" - Accuracy: 0.8202
	 Elapsed time 22.03909158706665
	 Fidelity: "0.8933" - Complexity: "3"
Split [2/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.9205
	 Epoch 1000: train accuracy: 0.9287
	 Epoch 1500: train accuracy: 0.9299
	 Epoch 2000: train accuracy: 0.9380
	 Epoch 2500: train accuracy: 0.9443
	 Epoch 3000: train accuracy: 0.9443
	 Epoch 3500: train accuracy: 0.9443
	 Epoch 4000: train accuracy: 0.9443
	 Epoch 4500: train accuracy:

	 Epoch 500: train accuracy: 0.9237
	 Epoch 1000: train accuracy: 0.9400
	 Epoch 1500: train accuracy: 0.9418
	 Epoch 2000: train accuracy: 0.9425
	 Epoch 2500: train accuracy: 0.9431
	 Epoch 3000: train accuracy: 0.9437
	 Epoch 3500: train accuracy: 0.9437
	 Epoch 4000: train accuracy: 0.9437
	 Epoch 4500: train accuracy: 0.9437
	 Epoch 5000: train accuracy: 0.9437
	 Model's accuracy: 0.8870
	 Model's accuracy: 0.8870
	 Class 1 - Global explanation: "stroke_flg & age_HIGH & ~afib_flg & ~sapsi_first_LOW & ~sapsi_first_HIGH" - Accuracy: 0.8701
	 Elapsed time 14.433847188949585
	 Fidelity: "0.9605" - Complexity: "5"
Consistency of explanations: 0.2600


,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,weights,0,afib_flg & resp_flg & ~stroke_flg,0.848315,0.820225,0.893258,3,0.26,22.039092
1,weights,1,stroke_flg & age_HIGH & sofa_first_LOW & ~afib...,0.876404,0.842697,0.887640,7,0.26,14.769521
2,weights,2,bmi_LOW & hgb_first_NORMAL & hr_1st_LOW & icu_...,0.837079,0.825843,0.898876,19,0.26,17.263793
3,weights,3,stroke_flg & ~sapsi_first_LOW & ~sapsi_first_H...,0.876404,0.853933,0.943820,4,0.26,17.304168
4,weights,4,resp_flg & ~stroke_flg & ~sapsi_first_LOW & ~w...,0.837079,0.769663,0.797753,4,0.26,15.136028
5,weights,5,stroke_flg & ~hgb_first_LOW & ~sapsi_first_HIGH,0.825843,0.820225,0.882022,3,0.26,14.199698
6,weights,6,resp_flg & ~stroke_flg & ~abg_count_NORMAL & ~...,0.819209,0.723164,0.734463,9,0.26,21.978889
7,weights,7,aline_flg & chf_flg & afib_flg & resp_flg & ab...,0.881356,0.841808,0.915254,90,0.26,1149.876298
8,weights,8,afib_flg & stroke_flg & abg_count_LOW & age_HI...,0.847458,0.841808,0.870056,90,0.26,1375.152574
9,weights,9,stroke_flg & age_HIGH & ~afib_flg & ~sapsi_fir...,0.887006,0.870056,0.960452,5,0.26,14.433847


# Psi network

In [8]:
method = 'psi'
need_pruning = True
relu = False
results_psi = c_to_y(method, need_pruning, relu, verbose=True)
results_psi

Split [1/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.8411
	 Epoch 1500: train accuracy: 0.8529
	 Epoch 2000: train accuracy: 0.8867
	 Epoch 2500: train accuracy: 0.8411
	 Epoch 3000: train accuracy: 0.8411
	 Epoch 3500: train accuracy: 0.8411
	 Epoch 4000: train accuracy: 0.8411
	 Epoch 4500: train accuracy: 0.8411
	 Epoch 5000: train accuracy: 0.8411
	 Model's accuracy: 0.8371
	 Model's accuracy: 0.8371
	 Class 1 - Global explanation: "(stroke_flg | bun_first_NORMAL | sapsi_first_HIGH)" - Accuracy: 0.8315
	 Elapsed time 8.920722484588623
	 Fidelity: "0.8483" - Complexity: "3"
Split [2/10]
	 Epoch 0: train accuracy: 0.1589
	 Epoch 500: train accuracy: 0.8411
	 Epoch 1000: train accuracy: 0.8411
	 Epoch 1500: train accuracy: 0.8586
	 Epoch 2000: train accuracy: 0.8899
	 Epoch 2500: train accuracy: 0.8411
	 Epoch 3000: train accuracy: 0.8411
	 Epoch 3500: train accuracy: 0.8411
	 Epoch 4000: train accuracy: 0.8411
	 Epoch 4500

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,psi,0,(stroke_flg | bun_first_NORMAL | sapsi_first_H...,0.837079,0.831461,0.848315,3,0.371429,8.920722
1,psi,1,(stroke_flg | abg_count_NORMAL),0.837079,0.814607,0.853933,2,0.371429,8.738159
2,psi,2,(stroke_flg | abg_count_HIGH | sapsi_first_HIGH),0.837079,0.842697,0.837079,3,0.371429,8.623687
3,psi,3,(stroke_flg | sapsi_first_HIGH),0.842697,0.842697,0.876404,2,0.371429,8.538700
4,psi,4,(stroke_flg | bun_first_NORMAL),0.842697,0.831461,0.842697,2,0.371429,8.668616
5,psi,5,(stroke_flg | bun_first_HIGH | chloride_first_...,0.842697,0.814607,0.825843,3,0.371429,8.461923
6,psi,6,(stroke_flg | bun_first_HIGH | chloride_first_...,0.836158,0.790960,0.819209,3,0.371429,8.508170
7,psi,7,(stroke_flg | bun_first_NORMAL | bun_first_HIGH),0.841808,0.847458,0.870056,3,0.371429,9.102114
8,psi,8,(stroke_flg | bun_first_NORMAL),0.841808,0.807910,0.819209,2,0.371429,8.546207
9,psi,9,(stroke_flg | chloride_first_HIGH | sapsi_firs...,0.841808,0.858757,0.870056,3,0.371429,10.023790


# Decision tree

In [9]:
method = 'tree'
need_pruning = False
relu = False
results_tree = c_to_y(method, need_pruning, relu, verbose=True)
results_tree

Split [1/10]
	 Model's accuracy: 0.8034
	 Class 1 - Global explanation: "(stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & wbc_first_NORMAL <= 0.50 & map_1st_LOW <= 0.50 & bun_first_LOW <= 0.50 & sofa_first_LOW > 0.50) | (stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & wbc_first_NORMAL <= 0.50 & map_1st_LOW <= 0.50 & bun_first_LOW > 0.50 & age_LOW <= 0.50 & hour_icu_intime_HIGH <= 0.50 & po2_first_NORMAL <= 0.50 & hr_1st_HIGH <= 0.50 & sofa_first_NORMAL > 0.50 & potassium_first_NORMAL <= 0.50 & day_icu_intime_num_NORMAL > 0.50 & service_num_HIGH > 0.50 & tco2_first_NORMAL > 0.50 & gender_num > 0.50 & hgb_first_HIGH <= 0.50) | (stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & wbc_first_NORMAL <= 0.50 & map_1st_LOW <= 0.50 & bun_first_LOW > 0.50 & age_LOW <= 0.50 & hour_icu_intime_HIGH <= 0.50 & po2_first_NORMAL <= 0.50 & hr_1st_HIGH <= 0.50 & sofa_first_NORMAL > 0.50 & potassium_first_NORMAL > 0.50 & al

Split [8/10]
	 Model's accuracy: 0.8136
	 Class 1 - Global explanation: "(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & liver_flg <= 0.50 & sapsi_first_HIGH <= 0.50 & sofa_first_NORMAL <= 0.50 & hr_1st_LOW > 0.50 & hour_icu_intime_NORMAL > 0.50 & day_icu_intime_num_LOW > 0.50 & aline_flg > 0.50) | (age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & liver_flg <= 0.50 & sapsi_first_HIGH <= 0.50 & sofa_first_NORMAL > 0.50 & hour_icu_intime_HIGH <= 0.50 & icu_los_day_NORMAL <= 0.50 & mal_flg <= 0.50 & copd_flg <= 0.50 & chloride_first_LOW <= 0.50 & map_1st_LOW <= 0.50 & age_LOW <= 0.50 & potassium_first_LOW <= 0.50 & day_icu_intime_num_HIGH > 0.50 & aline_flg <= 0.50 & tco2_first_LOW <= 0.50) | (age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_flg <= 0.50 & chf_flg <= 0.50 & liver_flg <= 0.50 & sapsi_first_HIGH <= 0.50 & sofa_first_NORMAL > 0.50 & hour_icu_intime_HIGH <= 0.50 & icu_los_day_NORMAL <= 0.50 & mal_flg <= 0.50 & copd_flg <= 0

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,tree,0,(stroke_flg <= 0.50 & age_HIGH <= 0.50 & resp_...,0.803371,0.803371,1.0,1363,0.815033,0.015621
1,tree,1,(stroke_flg <= 0.50 & age_HIGH <= 0.50 & sapsi...,0.803371,0.803371,1.0,1424,0.815033,0.016628
2,tree,2,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & age_N...,0.808989,0.808989,1.0,1345,0.815033,0.015625
3,tree,3,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & age_N...,0.780899,0.780899,1.0,1391,0.815033,0.015620
4,tree,4,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & sapsi...,0.786517,0.786517,1.0,1387,0.815033,0.015621
5,tree,5,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.848315,0.848315,1.0,1319,0.815033,0.015655
6,tree,6,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.762712,0.762712,1.0,1271,0.815033,0.015620
7,tree,7,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.813559,0.813559,1.0,1321,0.815033,0.022094
8,tree,8,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & resp_...,0.774011,0.774011,1.0,1353,0.815033,0.015626
9,tree,9,(age_HIGH <= 0.50 & stroke_flg <= 0.50 & sapsi...,0.774011,0.774011,1.0,1252,0.815033,0.015621


# BRL

In [10]:
method = 'brl'
need_pruning = False
relu = False
results_brl = c_to_y(method, need_pruning, relu, verbose=True)
results_brl

Split [1/10]
Labels (1598, 1)
Discretized features
Completed model 1/1!
Train_acc: 0.9, Val_acc: 0.0
	 Model's accuracy: 0.8596
	 Class 1 - Global explanation: "(stroke_flg & resp_flg & ~age_LOW & ~sapsi_first_LOW) | (stroke_flg & age_HIGH & ~age_LOW & ~sapsi_first_LOW)" - Accuracy: 0.8596
	 Elapsed time 482.5108571052551
	 Fidelity: "1.0000" - Complexity: "8"
Split [2/10]
Labels (1598, 1)
Discretized features
Completed model 1/1!
Train_acc: 0.9, Val_acc: 0.0
	 Model's accuracy: 0.8483
	 Class 1 - Global explanation: "(stroke_flg & ~renal_flg & ~age_LOW & ~potassium_first_LOW) | (stroke_flg & ~renal_flg & ~age_LOW & ~sapsi_first_LOW)" - Accuracy: 0.8483
	 Elapsed time 477.5066337585449
	 Fidelity: "1.0000" - Complexity: "8"
Split [3/10]
Labels (1598, 1)
Discretized features
Completed model 1/1!
Train_acc: 0.9, Val_acc: 0.0
	 Model's accuracy: 0.8820
	 Class 1 - Global explanation: "(stroke_flg & resp_flg & icu_los_day_NORMAL & ~sapsi_first_LOW & ~weight_first_NORMAL) | (stroke_flg & ag

,method,split,explanation,model_accuracy,explanation_accuracy,explanation_fidelity,explanation_complexity,explanation_consistency,elapsed_time
0,brl,0,(stroke_flg & resp_flg & ~age_LOW & ~sapsi_fir...,0.859551,0.859551,1.0,8,0.283333,482.510857
1,brl,1,(stroke_flg & ~renal_flg & ~age_LOW & ~potassi...,0.848315,0.848315,1.0,8,0.283333,477.506634
2,brl,2,(stroke_flg & resp_flg & icu_los_day_NORMAL & ...,0.882022,0.882022,1.0,20,0.283333,488.674345
3,brl,3,(stroke_flg & ~afib_flg & ~age_LOW & ~age_NORM...,0.853933,0.853933,1.0,40,0.283333,483.082235
4,brl,4,(stroke_flg & ~iv_day_1_HIGH & ~(age_LOW & ~ch...,0.853933,0.853933,1.0,17,0.283333,484.429098
5,brl,5,(stroke_flg & weight_first_LOW & ~copd_flg & ~...,0.848315,0.848315,1.0,8,0.283333,489.483012
6,brl,6,(stroke_flg & po2_first_LOW & sofa_first_HIGH ...,0.836158,0.836158,1.0,32,0.283333,476.141201
7,brl,7,(copd_flg & stroke_flg & resp_flg & weight_fir...,0.875706,0.875706,1.0,40,0.283333,502.090084
8,brl,8,(resp_flg & bun_first_NORMAL) | (bun_first_NOR...,0.881356,0.858757,1.0,24,0.283333,539.208452
9,brl,9,(stroke_flg & resp_flg & ~age_LOW & ~sapsi_fir...,0.875706,0.875706,1.0,10,0.283333,532.822168


# Summary

In [11]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_fidelity', 
        'explanation_complexity', 'elapsed_time', 'explanation_consistency']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

# BRL
df_mean = results_brl[cols].mean()
df_sem = results_brl[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_brl = pd.concat([df_mean, df_sem])
summary_brl.name = 'BRL'

summary = pd.concat([summary_pruning,
                     summary_weights, 
                     summary_psi, 
                     summary_tree,
                     summary_brl], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_mean,explanation_accuracy_mean,explanation_fidelity_mean,explanation_complexity_mean,elapsed_time_mean,explanation_consistency_mean,model_accuracy_sem,explanation_accuracy_sem,explanation_fidelity_sem,explanation_complexity_sem,elapsed_time_sem,explanation_consistency_sem
pruning,0.865987,0.850797,0.96228,5.1,13.319946,0.510000,0.003019,0.004809,0.008124,0.348010,0.303130,0.000000e+00
weights,0.853615,0.820942,0.87836,23.4,266.215391,0.260000,0.007811,0.013768,0.021187,11.200397,166.899079,1.850372e-17
psi,0.840091,0.828261,0.84628,2.6,8.813209,0.371429,0.000894,0.006595,0.006734,0.163299,0.148552,0.000000e+00
tree,0.795575,0.795575,1.00000,1342.6,0.016373,0.815033,0.007968,0.007968,0.000000,16.940550,0.000643,3.700743e-17
BRL,0.861499,0.859240,1.00000,20.7,495.594809,0.283333,0.005085,0.004582,0.000000,4.077172,7.128592,1.850372e-17


In [12]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))